# Extract Locations 

In [3]:
import pandas as pd
import PyPDF2 as p2
import re
import os
import pickle
import json
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
from PyPDF2 import PdfFileReader, PdfFileWriter

from PIL import Image
# import ghostscript
# from pdf2image import convert_from_path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import location_extractor
import re 

import re
#import math
import collections

import gis_converter
import util
import html_analyzer

In [5]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 4 - Final\Indices\Index 2 - PDFs for Major Projects with ESAs.csv')

ngtl_2021 = index2[index2['Link to Application'].str.contains("https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3575553")]
ngtl_2021


pdf_names = ngtl_2021['DataID']

path = "F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_"
pkl_paths = [path + str(x) + '.pkl' for x in pdf_names]
pkl_paths

,Unnamed: 0,file_name,application_name,esa_folder_link,DocumentID,DataID,esa_download_link,DataID_pdf,Section,Application filing date,...,Company Name,Consultant Name,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location
1695,1695,A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q3,3579528,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3579528.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1696,1696,A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q4,3578647,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3578647.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1697,1697,A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q5,3581069,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3581069.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1698,1698,A92619-13 13 ESA Section 11 - A6F4Q6,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q6,3578648,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3578648.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1699,1699,A92619-14 14 ESA Sections 12 to 16 - A6F4Q7,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q7,3579739,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3579739.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1700,1700,A92619-15 15 ESA Sections 17 to 26 - A6F4Q8,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q8,3579849,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3579849.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1701,1701,A92619-16 16 ESA Appendix A EPP - A6F4Q9,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A6F4Q9,3579311,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,3579311.pdf,52,2018-06-20,...,NOVA Gas Transmission Ltd.,"Wood Environment & Infrastructure Solutions, S...",Applied,NGTL s. 52 application - 2021 NGTL System Expa...,NaN,NaN,Application for 2021 NGTL System Expansion Pro...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,pending,TBD
1702,1702,A92619-17 17 ESA Appendix B EAS-Part A - A6F4R0,2018-06-20 Application for 2021 NGTL System Ex...,https://apps.cer-rec.gc.ca/RE

['F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579528.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578647.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3581069.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578648.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579739.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579849.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579311.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3581159.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3580072.pkl',
 'F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3580186.pkl']

# Extract Locations 

In [6]:
def _add_to_hist(locations_container, locations, page):
    """
        Adds all locations in a list of locations to a dictionary 
        in a histogram-like fashion (if the location already exists, 
        just increment the count and add the page number, don't 
        introduce another location in the dictionary.)
    """
    for location in locations:
        if location.text in locations_container:
            locations_container[location.text]['count'] += 1
            locations_container[location.text]['pages'].append(page)
        else:
            locations_container[location.text] = {'count': 1,
                                                  'pages': [page],
                                                  'tuple': location}
            try:
                locations_container[location.text]['latitude-longitude'] = location.lat_long
            except AttributeError:
                pass

In [7]:
# initialize variables to count number of documents with a certain location
# format
# num_lat_long = 0
# num_dls = 0
# num_nts = 0
# num_mlv = 0


lat_long_tags = {}
dls_tags = {}
nts_tags = {}
mlv_tags = {}

num_files = 0
num_files_tagged = 0

# container for results to be converted to JSON later
result = {}
# list of files that failed to open or errored when running the code
bad_files = []


for x in pkl_paths:
    try:
        with open(x, 'rb') as f:
            data = pickle.load(f)
            soup = BeautifulSoup(data['content'], 'lxml')
            pages = soup.find_all('div', attrs={'class': 'page'})
            print("Processing File: {} with {} number of pages".format(x,len(pages)))
            for b, p in enumerate(pages):
                
                # find locations on the page and add to the dictionary for the
                # location format
                lat_long, dls, nts, mlv = html_analyzer.find_location_tags(page)
                    
                _add_to_hist(lat_long_tags, lat_long, page_number)
                _add_to_hist(dls_tags, dls, page_number)
                _add_to_hist(nts_tags, nts, page_number)
                _add_to_hist(mlv_tags, mlv, page_number)
                
            
            """
                Refactor the histogram-like dictionaries to lists so they can
                be converted effectively to json
            """
            lat_long_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'degrees-north': tag['tuple'].N,
                        'degrees-west': tag['tuple'].W
                    }
                for tag_text, tag in lat_long_tags.items()]
            
            dls_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'legal_subdivision': tag['tuple'].legal_subdivision,
                        'section': tag['tuple'].section,
                        'township': tag['tuple'].township,
                        'range': tag['tuple'].range,
                        'meridian': tag['tuple'].meridian,
                        
                        'latitude-longitude': {
                            'text': tag['latitude-longitude'].text,
                            'degrees-north': tag['latitude-longitude'].N,
                            'degrees-west': tag['latitude-longitude'].W
                        }
                    }
                for tag_text, tag in dls_tags.items()]
            
            nts_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'quarter_unit': tag['tuple'].quarter_unit,
                        'unit': tag['tuple'].unit,
                        'block': tag['tuple'].block,
                        'series_number': tag['tuple'].series_number,
                        'map_area': tag['tuple'].map_area,
                        'map_sheet': tag['tuple'].map_sheet,
                        
                        'latitude-longitude': {
                            'text': tag['latitude-longitude'].text,
                            'degrees-north': tag['latitude-longitude'].N,
                            'degrees-west': tag['latitude-longitude'].W
                        }
                    }
                for tag_text, tag in nts_tags.items()]
            
            mlv_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'valve': tag['tuple'].valve,
                        'offset': tag['tuple'].offset
                    }
                for tag_text, tag in mlv_tags.items()]
            
            # store the result of location extraction on this document in the
            # main result dictionary
            result[''.join(FILENAME.split('.html')[:-1])] = {
                    'latitude-longitude': lat_long_tags,
                    'dominion-land-survey': dls_tags,
                    'national-topographic-system': nts_tags,
                    'mainline-valve': mlv_tags
                    }
            
            if len(lat_long_tags) > 0 or len(dls_tags) > 0 or len(nts_tags) > 0 or len(mlv_tags) > 0:
                num_files_tagged += 1
            
            num_files += 1
    except:
        bad_files.append(x)
        print('Error in ' + x)
        

        
print("Total Number of Files processed: {}".format(num_files))
print("Files with Locations: {}".format(num_files))
print("Bad Files Pages {}".format(len(bad_files)))



Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579528.pkl with 112 number of pages
Error in F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579528.pkl
Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578647.pkl with 321 number of pages
Error in F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578647.pkl
Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3581069.pkl with 114 number of pages
Error in F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3581069.pkl
Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578648.pkl with 236 number of pages
Error in F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3578648.pkl
Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579739.pkl with 362 number of page

In [8]:
# initialize variables to count number of documents with a certain location
# format
# num_lat_long = 0
# num_dls = 0
# num_nts = 0
# num_mlv = 0




num_files = 0
num_files_tagged = 0

# container for results to be converted to JSON later
result = {}
# list of files that failed to open or errored when running the code
bad_files = []


for x in pkl_paths:
    with open(x, 'rb') as f:
        
        data = pickle.load(f)
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        print("Processing File: {} with {} number of pages".format(x,len(pages)))
        
        lat_long_tags = {}
        dls_tags = {}
        nts_tags = {}
        mlv_tags = {}
        
        def _add_to_hist(locations_container, locations, page):
            """
                Adds all locations in a list of locations to a dictionary 
                in a histogram-like fashion (if the location already exists, 
                just increment the count and add the page number, don't 
                introduce another location in the dictionary.)
            """
            for location in locations:
                if location.text in locations_container:
                    locations_container[location.text]['count'] += 1
                    locations_container[location.text]['pages'].append(page)
                else:
                    locations_container[location.text] = {
                        'count': 1,
                        'pages': [page],
                        'tuple': location}
                try:
                    locations_container[location.text]['latitude-longitude'] = location.lat_long
                except AttributeError:
                    pass
        
        
        for b, p in enumerate(pages):
                  
                # find locations on the page and add to the dictionary for the
                # location format
            page_number = b+ 1
            lat_long, dls, nts, mlv = html_analyzer.find_location_tags_from_text(str(p),int(b)+1)
                    
            _add_to_hist(lat_long_tags, lat_long, page_number)
            _add_to_hist(dls_tags, dls, page_number)
            _add_to_hist(nts_tags, nts, page_number)
            _add_to_hist(mlv_tags, mlv, page_number)
                
            
            """
                Refactor the histogram-like dictionaries to lists so they can
                be converted effectively to json
            """
        lat_long_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'degrees-north': tag['tuple'].N,
                        'degrees-west': tag['tuple'].W
                    }
            for tag_text, tag in lat_long_tags.items()]
            
        dls_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'legal_subdivision': tag['tuple'].legal_subdivision,
                        'section': tag['tuple'].section,
                        'township': tag['tuple'].township,
                        'range': tag['tuple'].range,
                        'meridian': tag['tuple'].meridian,
                        
                        'latitude-longitude': {
                            'text': tag['latitude-longitude'].text,
                            'degrees-north': tag['latitude-longitude'].N,
                            'degrees-west': tag['latitude-longitude'].W
                        }
                    }
            for tag_text, tag in dls_tags.items()]
            
        nts_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'quarter_unit': tag['tuple'].quarter_unit,
                        'unit': tag['tuple'].unit,
                        'block': tag['tuple'].block,
                        'series_number': tag['tuple'].series_number,
                        'map_area': tag['tuple'].map_area,
                        'map_sheet': tag['tuple'].map_sheet,
                        
                        'latitude-longitude': {
                            'text': tag['latitude-longitude'].text,
                            'degrees-north': tag['latitude-longitude'].N,
                            'degrees-west': tag['latitude-longitude'].W
                        }
                    }
            for tag_text, tag in nts_tags.items()]
            
        mlv_tags = [
                    {
                        'text': tag_text,
                        'count': tag['count'],
                        'pages': tag['pages'],
                        
                        # format-specific fields
                        'valve': tag['tuple'].valve,
                        'offset': tag['tuple'].offset
                    }
            for tag_text, tag in mlv_tags.items()]
            
            # store the result of location extraction on this document in the
            # main result dictionary
        result[x.split("_")[1].split('.')[0]] = {
                    'latitude-longitude': lat_long_tags,
                    'dominion-land-survey': dls_tags,
                    'national-topographic-system': nts_tags,
                    'mainline-valve': mlv_tags
                    }
            
        if len(lat_long_tags) > 0 or len(dls_tags) > 0 or len(nts_tags) > 0 or len(mlv_tags) > 0:
            num_files_tagged += 1
            
        num_files += 1
    
        

        
print("Total Number of Files processed: {}".format(num_files))
print("Files with Locations: {}".format(num_files))
print("Bad Files Pages {}".format(len(bad_files)))



Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_3579528.pkl with 112 number of pages
format of Tika_text <class 'str'>
format of number of page <class 'int'>


'\n                Refactor the histogram-like dictionaries to lists so they can\n                be converted effectively to json\n            '

format of Tika_text <class 'str'>
format of number of page <class 'int'>


'\n                Refactor the histogram-like dictionaries to lists so they can\n                be converted effectively to json\n            '

format of Tika_text <class 'str'>
format of number of page <class 'int'>


...

format of Tika_text <class 'str'>
format of number of page <class 'int'>


'\n                Refactor the histogram-like dictionaries to lists so they can\n                be converted effectively to json\n            '

format of Tika_text <class 'str'>
format of number of page <class 'int'>


'\n                Refactor the histogram-like dictionaries to lists so they can\n                be converted effectively to json\n            '

Total Number of Files processed: 10
Files with Locations: 10
Bad Files Pages 0


In [9]:
OUTPUT_FOLDER = "C://Users//T1Vibudh//Desktop//RF - old Trining//"

In [10]:
with open(OUTPUT_FOLDER + 'locations-raw.txt', 'w+', encoding='utf-8') as output:
    output.write(json.dumps(result, indent=2))

# save names of files that failed to open or errored when running this code
# into a text log in the output folder
with open(OUTPUT_FOLDER + 'bad-files.txt', 'w+', encoding='utf-8') as output:
    output.write('\n'.join(bad_files))


#%% some cleaning of the locations (remove outliers)
# set the working folder to the output folder of the previous chunk of code
WORK_FOLDER = OUTPUT_FOLDER



206270

0

In [11]:
# save the summary of how many files were identified with each location
# format into a summary text file
with open(OUTPUT_FOLDER + 'summary-raw.txt', 'w+', encoding='utf-8') as output:
    output.write('{} out of {} files were tagged ({}%)\n'.format(num_files_tagged, num_files, 100 * float(num_files_tagged) / num_files))
    output.write('{} out of {} files contained latitude-longitude coordinates ({}%)\n'.format(num_lat_long, num_files, 100 * float(num_lat_long) / num_files))
    output.write('{} out of {} files contained Dominion Land Survey coordinates ({}%)\n'.format(num_dls, num_files, 100 * float(num_dls) / num_files))
    output.write('{} out of {} files contained National Topographic System coordinates ({}%)\n'.format(num_nts, num_files, 100 * float(num_nts) / num_files))
    output.write('{} out of {} files contained Mainline Valve locations ({}%)\n'.format(num_mlv, num_files, 100 * float(num_mlv) / num_files))


38

NameError: name 'num_lat_long' is not defined

In [38]:
for nam in pkl_paths:
    

F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_268706.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_268709.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_268712.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269018.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269021.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269024.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269027.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269030.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269033.pkl
F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_268930.pkl


In [ ]:
names = 'F:\Environmental Baseline Data\Version 4 - Final\Tika - Pickles\tikaxml_268933.pkl'

names.split("_")[1].split('.')[0]

In [14]:
v = "vibudh"
v.split("i")

['v', 'budh']

In [8]:
for x in pkl_paths:
    with open(x, 'rb') as f:
        
        data = pickle.load(f)
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        print("Processing File: {} with {} number of pages".format(x,len(pages)))
        for b, p in enumerate(pages):
            print("Page Number {}. Type Text: {}".format(b+1 ,type(str(p))))

Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_268706.pkl with 107 number of pages
Page Number 1. Type Text: <class 'str'>
Page Number 2. Type Text: <class 'str'>
Page Number 3. Type Text: <class 'str'>
Page Number 4. Type Text: <class 'str'>
Page Number 5. Type Text: <class 'str'>
Page Number 6. Type Text: <class 'str'>
Page Number 7. Type Text: <class 'str'>
Page Number 8. Type Text: <class 'str'>
Page Number 9. Type Text: <class 'str'>
Page Number 10. Type Text: <class 'str'>
Page Number 11. Type Text: <class 'str'>
Page Number 12. Type Text: <class 'str'>
Page Number 13. Type Text: <class 'str'>
Page Number 14. Type Text: <class 'str'>
Page Number 15. Type Text: <class 'str'>
Page Number 16. Type Text: <class 'str'>
Page Number 17. Type Text: <class 'str'>
Page Number 18. Type Text: <class 'str'>
Page Number 19. Type Text: <class 'str'>
Page Number 20. Type Text: <class 'str'>
Page Number 21. Type Text: <class 'str'>
Page Number 22. Type Te

Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269021.pkl with 15 number of pages
Page Number 1. Type Text: <class 'str'>
Page Number 2. Type Text: <class 'str'>
Page Number 3. Type Text: <class 'str'>
Page Number 4. Type Text: <class 'str'>
Page Number 5. Type Text: <class 'str'>
Page Number 6. Type Text: <class 'str'>
Page Number 7. Type Text: <class 'str'>
Page Number 8. Type Text: <class 'str'>
Page Number 9. Type Text: <class 'str'>
Page Number 10. Type Text: <class 'str'>
Page Number 11. Type Text: <class 'str'>
Page Number 12. Type Text: <class 'str'>
Page Number 13. Type Text: <class 'str'>
Page Number 14. Type Text: <class 'str'>
Page Number 15. Type Text: <class 'str'>
Processing File: F:/Environmental Baseline Data/Version 4 - Final/Tika - Pickles/tikaxml_269024.pkl with 16 number of pages
Page Number 1. Type Text: <class 'str'>
Page Number 2. Type Text: <class 'str'>
Page Number 3. Type Text: <class 'str'>
Page Number 4. Type Text: 

In [ ]:
#open pickle file to extact strings startswith figure or title
#7 min execution time

start = time.time()
file_name = []
page_number = []
table_title = []
table_title_next = []




exceptions_list = ['...', 'Table of Content', 'TABLE OF CONTENTS', 'Table des matières', 'TABLE DES MATIÈRES'] 
last_ptag_starts_with_key = 0
for x in all_paths:
    with open(x, 'rb') as f:
        data = pickle.load(f)
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        for b, p in enumerate(pages):
            pages_text = [x.text for x in p.find_all('p')]
            for y in pages_text:
                if last_ptag_starts_with_key ==1:
                    table_title_next.append(y.replace('\n',' ').replace('\r',''))
                last_ptag_starts_with_key = 0
                
                ytest = y.replace('\n','').replace('\r','')
                ytest = ytest.split(" ")
                ytest_no_blanks = ""
                for ytes in ytest:
                    if len(ytes) > 2:
                        ytest_no_blanks = ytest_no_blanks + ytes + " "
                        
                if ytest_no_blanks.startswith(('Table', 'TABLE', 'FIGURE', 'FIGURES', 'Figure', 'Figures', 'IMAGE', 'IMAGES', 
                                               'Image', 'Images', 'PHOTO', 'PHOTOS', 'Photo', 
                                               'Photos')) and not any(substring in y for substring in exceptions_list):
                    file_name.append(x)
                    table_title.append(y.replace('\n','').replace('\r',''))
                    page_number.append(b + 1)
                    last_ptag_starts_with_key = 1
#                   
print('Time: ', time.time() - start)

In [71]:
a = re.findall("Vi", "ViVibudhViSinghVi")
print(a)

['Vi', 'Vi', 'Vi', 'Vi']
